In [ ]:
from helper import *

path='../reports/corpus.xlsx'

description = get_column_values(path, 'descriptions_measures', 'A')


targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_df = similarities_df(docs=description,group=targets,encoding_method='tfidf')

similarities_df.head()

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

processed_text = preprocess_text(Q1)



w2v = api.load("word2vec-google-news-300")
vocab = w2v.index_to_key
words = Q1.split()


count = sum(1 for word in processed_text if word in w2v)
existing_words = [word for word in processed_text if word in w2v]
difference = len(words) - count

pr, orig = get_unique_items(words, processed_text)

pr

**Save BOW models**

In [2]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = '/models/bow_model.joblib'
version_tfidf = '/models/tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)
ngram_range = (3,3)

tm.build_bow_model(preprocessing=False, ngram_range=ngram_range)
tm.build_tfidf_model(preprocessing=False, ngram_range=ngram_range)

tm.save_models('trigrams_')

Bag-of-words model saved to trigrams_bow_model.joblib
TF-IDF model saved to trigrams_tfidf_model.joblib


**Save yake and rake models**

In [3]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
mode = 'yake'

ykw_corpus = build_kw_corpus(mode=mode, descriptions=descriptions, targets=targets)

In [4]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
mode = 'rake'

rkw_corpus = build_kw_corpus(mode=mode, descriptions=descriptions, targets=targets)

In [6]:
ngram_range = (1,1)

yake_bow_model = CountVectorizer(ngram_range=ngram_range)
yake_bow_model.fit_transform(ykw_corpus)

rake_bow_model = CountVectorizer(ngram_range=ngram_range)
rake_bow_model.fit_transform(rkw_corpus)

<71x123 sparse matrix of type '<class 'numpy.int64'>'
	with 381 stored elements in Compressed Sparse Row format>

In [7]:
joblib.dump(yake_bow_model, './models/ykw_bow_model.joblib')
joblib.dump(rake_bow_model, './models/rkw_bow_model.joblib')

['./models/rkw_bow_model.joblib']

In [8]:
yake_tfidf_model = TfidfVectorizer(ngram_range=ngram_range)
yake_tfidf_model.fit_transform(ykw_corpus)

rake_tfidf_model = TfidfVectorizer(ngram_range=ngram_range)
rake_tfidf_model.fit_transform(rkw_corpus)

<71x123 sparse matrix of type '<class 'numpy.float64'>'
	with 381 stored elements in Compressed Sparse Row format>

In [9]:
joblib.dump(yake_tfidf_model, './models/ykw_tfidf_model.joblib')
joblib.dump(rake_tfidf_model, './models/rkw_rfidf_model.joblib')

['./models/rkw_rfidf_model.joblib']

**Testing W2V models**

In [61]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
preprocessing = False
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = 'preprocessedbow_model.joblib'
version_tfidf = 'preprocessedtfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)
n = 10
Q8 = 'running settlements expected monthly payments at the end of the examined month actual monthly payments'
scores = get_similarity_scores(text=descriptions[0], documents=targets, encoding_method='word2vec',
                               version=version_w2v, tm=tm, preprocessing=preprocessing )


scores

{'Total payments amount so far': 0.6399805,
 'Application status at the end of the preceding observation period': 0.63616157,
 'Application status in previous month': 0.6337284,
 'Payment prediction for stray payers, not in active settlement': 0.62978095,
 'Last rejection date of all applications within the observation perion': 0.5950874,
 'Amount paid within the observation period': 0.58748496,
 'Settlement duration more than 109 months': 0.57428753,
 'Status of the last submitted application within the observation period.': 0.5735114,
 'External agency (DCA or legal office) assigned on previous observation point': 0.56485486,
 'Application bucket in the previous observation period': 0.5552512,
 'Payment prediction for settlement payers': 0.55217916,
 'Application type in previous observation period': 0.5429492,
 'Application type in the observation period': 0.538208,
 'Settlement duration 7 to 12 months (included)': 0.5315263,
 'Settlement duration to 6 months (included)': 0.52221775

In [6]:
import gensim.downloader as api

def w2v_vec(text,w2v):
    text = text.split()
    embeddings = [w2v[word] for word in text if word in w2v]
    sentence_emb = sum(embeddings)/len(embeddings)
    return sentence_emb

def cos_sim(t1,t2,w2v):
  v1 = w2v_vec(t1,w2v)
  v2 = w2v_vec(t2,w2v)
  score = cosine_similarity([v1], [v2])[0][0]
  return score

w2v = api.load("word2vec-google-news-300")
Q8 = 'running settlements expected monthly payments at the end of the examined month actual monthly payments'
target_sample = 'Last rejection date of all applications within the observation perion'


sim = cos_sim(Q8,target_sample,w2v)
sim






0.4773875

In [9]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
w2v = 'word2vec-google-news-300'

dict_desc = percentage_of_words_in_word2vec_vocabulary(w2v,descriptions)
dict_tar = percentage_of_words_in_word2vec_vocabulary(w2v,targets)


In [22]:
#dict_desc
dict_tar
w2v = api.load('word2vec-google-news-300')
l = 'QC customer code'.split()

words_found = 0
for word in l:
    if word in w2v:
        words_found += 1

words_found

'QC' in w2v

True

# **Keywords**
**RAKE**

In [34]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
language = 'en'
library = 'rake'
max_kw = 20
max_length = 5
min_length = 1
stopwords = None#['create','report', 'earth', 'portfolio']

rake_kw, rake_kw_sentence = kw_extraction(library=library, text=descriptions[0],
                                          max_kw=max_kw, max_length=max_length, min_length=min_length,stopwords=stopwords)




In [35]:
rake_kw

['actual monthly payments per dca',
 'expected monthly payments',
 'september 2020',
 'running settlements',
 'examined month',
 'earth portfolio',
 'report',
 'provide',
 'number',
 'end',
 'displays',
 'accounts']

In [19]:
rake_kw_sentence

'median duration entry principal earth portfolio approved amount application type average days average written shows report report produced number month create balance balance approval'

**YAKE**

In [38]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
language = 'en'
library = 'yake'
max_length = 5 #for rake
min_length = 1 #for rake
max_kw = 20
dedupLim = 0.8
n = 3
stopwords = ['their','displays','report', 'that', 'the',
             'settlement', 'earth', 'portfolio', 'and']

yake_kw, yake_kw_sentence = kw_extraction(library=library, text=descriptions[5],
                                          max_kw=max_kw, max_length=max_length, n=n,
                                          min_length=min_length,stopwords=stopwords, dedupLim=dedupLim)

In [39]:
yake_kw

['applications submitted during',
 'for September',
 'per DCA',
 'during the month',
 'principal and balance',
 'application type',
 'shows the number',
 'submitted during',
 'approved amount',
 'entry principal',
 'applications submitted',
 'DCA and application',
 'September',
 'DCA',
 'Create',
 'month',
 'balance',
 'for',
 'per',
 'type']

In [14]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
preprocessing = False
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = 'preprocessedbow_model.joblib'
version_tfidf = 'preprocessedtfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)
n = 10
scores = get_similarity_scores(text=yake_kw_sentence, documents=targets, encoding_method='word2vec',
                               version=version_w2v, tm=tm, preprocessing=preprocessing )


scores

{'Last rejection date of all applications within the observation perion': 0.62871206,
 'Settlement duration more than 109 months': 0.61768234,
 'Settlement duration 7 to 12 months (included)': 0.60358554,
 'Settlement duration 13 to 36 months (included)': 0.6034324,
 'Settlement duration 37 to 72 months (included)': 0.6034324,
 'Settlement duration 73 to 108 months (included)': 0.6034324,
 'Settlement duration to 6 months (included)': 0.60212064,
 'Payment prediction for stray payers, not in active settlement': 0.6012644,
 'Status of the last submitted application within the observation period.': 0.59962094,
 'Approved amount': 0.591117,
 'Application status in previous month': 0.5853797,
 'Application type in the observation period': 0.56741595,
 'Application type in previous observation period': 0.5660828,
 'Total payments amount so far': 0.56281215,
 'Application status at the end of the preceding observation period': 0.5568165,
 'Payment prediction for settlement payers': 0.5420778

In [1]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
'''language = 'en'
library = 'yake'
max_kw = 10
dedupLim = 0.9
stopwords = ['report','settlement', 'earth', 'portfolio']'''
kwe = KeywordExtractor()

ykws = kwe(mode='yake', text=descriptions[0])
rkws = kwe(mode='rake', text=descriptions[0])

rkws


c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'expected monthly payments september 2020 running settlements examined month earth portfolio'

**Similarity metrics**

In [4]:
from similarity import *


string_a = 'Amount paid within the observation period'
string_b = 'Hello Word!'

algs = textdistance.algorithms
algs.levenshtein.distance(string_a, string_b)
# Distance equals 1.0
algs.levenshtein.similarity(string_a, string_b)
# Similarity equals 11
algs.ratcliff_obershelp.normalized_similarity(string_a, string_b)
# Similarity Normalized equals 0.9166666666666666


0.11538461538461542

In [8]:
from similarity import *
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
algorithm = 'ratcliff_obershelp'
mode = 'seq'
string_a = descriptions[0]
string_b = 'Amount paid within the observation period'
string_c = 'Balance on arrangement creation'

dsc = DocumentSimilarityCalculator(algorithm=algorithm, mode=mode)
dsc.calculate_similarity(string_a, string_c)



0.15209125475285168

In [59]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
algorithm = 'cosine'
mode = 'token'

scores = get_similarity_scores_td(text=descriptions[0], documents=targets,
                                  algorithm=algorithm, mode=mode)

In [60]:
scores

{'Status of the last submitted application within the observation period.': 0.553203834728774,
 'External agency (DCA or legal office) assigned on previous observation point': 0.5346967694017905,
 'Last rejection date of all applications within the observation perion': 0.5295491373381438,
 'Application status at the end of the preceding observation period': 0.5293131528623707,
 'Payment prediction for stray payers, not in active settlement': 0.5127680123435535,
 'Bucket class for active arrangements or cancelled status': 0.4649838442026326,
 'Application bucket in the previous observation period': 0.45992630452640504,
 'External agency (DCA or legal office) currently assigned': 0.4562105641233377,
 'Application type in previous observation period': 0.4405192693921527,
 'Application type in the observation period': 0.4254814716983816,
 'Amount paid within the observation period': 0.4203857013874692,
 'Payment prediction for settlement payers': 0.4152273992686999,
 'Settlement duration 7

**Synonyms**

In [2]:
import spacy

nlp = spacy.load("en_core_web_md")
word = nlp("happy")

synonyms = []
for syn in word:
    for lemma in syn.lemma_.split("|"):
        synonyms.append(lemma)

print(set(synonyms))


{'happy'}


In [7]:
from nltk.corpus import wordnet as wn

synonyms = []
for syn in wn.synsets('month'):
    for lemma in syn.lemmas():
        synonyms.append(lemma.name())

print(set(synonyms))

{'calendar_month', 'month'}


**Metrics**

In [9]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
model = 'word2vec'
preprocessing = False
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = './models/trigrams_bow_model.joblib'
version_tfidf = './models/trigrams_tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v)
metric1 = 'precision'
metric2 = 'recall'
n = 10

pr = evaluation_metrics(metric=metric1,values=descriptions, docs=targets, encoding_method=model,
               version=version_w2v, tm=tm, preprocessing=preprocessing, path=path, n=n)

rec = evaluation_metrics(metric=metric2, values=descriptions, docs=targets, encoding_method=model,
               version=version_w2v, tm=tm, preprocessing=preprocessing, path=path, n=n)

In [11]:
rec

pr

{'Q1': 10.0,
 'Q2': 20.0,
 'Q3': 10.0,
 'Q4': 30.0,
 'Q5': 'One or both of the lists is empty',
 'Q6': 0.0,
 'Q7': 0.0,
 'Q8': 0.0,
 'Q9': 0.0}

In [41]:
from helper import *

word2vec_version_fin_path = 'C:/Users/avitsas/Documents/VECTORS/word.w2v.200.bin'
sentence = 'Provide the number of customers with active and running settlements of types Settlement, Preappoved, Resch to Sett, Out of Mandate Had Settlement and null on the Earth portfolio, \
    the corresponding number of applications, the initial settlement amount, discount amount and future instalments per asset class and DCA, for September 2020.'
sentence = preprocess_text(sentence)
embeddings = []
embedding_model = KeyedVectors.load_word2vec_format(word2vec_version_fin_path, binary=True)

for word in sentence:
    if word in embedding_model:
        embeddings.append(embedding_model.get_vector(word))
if len(embeddings) > 0:
    sentence_embedding = sum(embeddings) / len(embeddings)





In [3]:
save_df_to_excel(evaluation_df, '../reports/results/metrics5.xlsx', 'r_precision')

c:\Users\avitsas\Documents\thesis-backup\thesis\code\helper.py:86: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
